In [10]:
import numpy as np
import time
import math
import pandas as pd
import itertools
import requests
from bs4 import BeautifulSoup
import sympy
from sympy.abc import a,b,c,d,e,f,g,h,i,j,k

In [11]:
print("Puzzle")
print("~~~~~~")
url='https://www.janestreet.com/puzzles/figurine-figuring/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
x =[text for text in soup.body.stripped_strings]
print("".join(x[7:]))

from sympy.abc import a,b,c,d,e,f,g,h,i,j,k


Puzzle
~~~~~~
Jane received 78 figurines as gifts this holiday season:  12 drummers drumming, 11 pipers piping, 10 lords a-leaping, etc., down to 1 partridge in a pear tree.   They are all mixed together in a big bag.  She agrees with her friend Alex that this seems like too many figurines for one person to have, so she decides to give some of her figurines to Alex.   Jane will uniformly randomly pull figurines out of the bag one at a time until she pulls out the partridge in a pear tree, and will give Alex all of the figurines she pulled out of the bag (except the partridge, that’s Jane’s favorite).Ifnis the maximum number of any one type of ornament that Alex gets, what is the expected value ofn, to seven significant figures?


In [12]:
# Generating function to work out the number of permutations for the various pulls.
# 

terms = [ (1+a),
          (1+a+a**2),
          (1+a+a**2+a**3),
          (1+a+a**2+a**3+a**4),
          (1+a+a**2+a**3+a**4+a**5),
          (1+a+a**2+a**3+a**4+a**5+a**6),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9+a**10),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9+a**10+a**11),
          (1+a+a**2+a**3+a**4+a**5+a**6+a**7+a**8+a**9+a**10+a**11+a**12),
          ]
    

In [25]:

perms=[]

perms.append(sympy.Poly(terms[0]**11).monoms())

for i in range(1,12):
    p = sympy.Poly(1,a)
    
    for j in range(1,12):
        p *= sympy.Poly(terms[min(i,j)])
    perms.append(p.monoms())
    
for i in perms:
    print(len(i),i)


12 [(11,), (10,), (9,), (8,), (7,), (6,), (5,), (4,), (3,), (2,), (1,), (0,)]
23 [(22,), (21,), (20,), (19,), (18,), (17,), (16,), (15,), (14,), (13,), (12,), (11,), (10,), (9,), (8,), (7,), (6,), (5,), (4,), (3,), (2,), (1,), (0,)]
33 [(32,), (31,), (30,), (29,), (28,), (27,), (26,), (25,), (24,), (23,), (22,), (21,), (20,), (19,), (18,), (17,), (16,), (15,), (14,), (13,), (12,), (11,), (10,), (9,), (8,), (7,), (6,), (5,), (4,), (3,), (2,), (1,), (0,)]
42 [(41,), (40,), (39,), (38,), (37,), (36,), (35,), (34,), (33,), (32,), (31,), (30,), (29,), (28,), (27,), (26,), (25,), (24,), (23,), (22,), (21,), (20,), (19,), (18,), (17,), (16,), (15,), (14,), (13,), (12,), (11,), (10,), (9,), (8,), (7,), (6,), (5,), (4,), (3,), (2,), (1,), (0,)]
50 [(49,), (48,), (47,), (46,), (45,), (44,), (43,), (42,), (41,), (40,), (39,), (38,), (37,), (36,), (35,), (34,), (33,), (32,), (31,), (30,), (29,), (28,), (27,), (26,), (25,), (24,), (23,), (22,), (21,), (20,), (19,), (18,), (17,), (16,), (15,), (14,)

In [ ]:
start = time.perf_counter()
x = []
GF = sympy.Poly(func(a))

for i in range(12):
    x.appendGF.coeffs()
    
stop =  time.perf_counter()
print('\n Solution took {:0.4f} seconds\n'.format((stop-start)))

In [ ]:
help(GF)